## 信號量Semaphore
---
信號量Semaphore是一種用於控制對共享資源訪問的同步原語。它可以用來限制同時訪問某個特定資源的執行緒數量。Semaphore的計數器表示當前可用資源的數量，當計數器大於0時，執行緒可以訪問資源；當計數器為0時，執行緒需要等待。

In [12]:
import threading
import time

# 網頁爬蟲類別 (目的:控制同時工作爬蟲的數量，避免被封IP)
class HtmlSpider(threading.Thread):
    def __init__(self, name, semaphore, lock):
        super().__init__()
        self.name = name
        self.semaphore = semaphore
        self.lock = lock

    def run(self):
        time.sleep(2)
        with self.lock:  # 避免不同thread同時print的干擾狀況
            print(f"{self.name} got html text success in ", time.strftime("%H:%M:%S", time.localtime()))
        self.semaphore.release() # 釋放信號量 (將計數器加1)

# 獲取網址內容類別
class UrlProducer(threading.Thread):
    def __init__(self, semaphore, lock):
        super().__init__()
        self.semaphore = semaphore
        self.lock = lock
        self.spiders = []

    def run(self):
        for i in range(20):
            with self.lock:
                print(f"UrlProducer-{i} called new htmlSpider")
            self.semaphore.acquire()  # 獲取信號量 (將計數器減1)
            spider = HtmlSpider(f"htmlSpider-{i}", self.semaphore, self.lock)
            self.spiders.append(spider)
            spider.start()
        for spider in self.spiders:
            spider.join()  # 等待所有HtmlSpider執行緒結束

semaphore = threading.Semaphore(3)  # 限制同時訪問資源的執行緒數量為3
lock = threading.Lock()
urlProducer = UrlProducer(semaphore, lock)
urlProducer.start()
urlProducer.join()

UrlProducer-0 called new htmlSpider
UrlProducer-1 called new htmlSpider
UrlProducer-2 called new htmlSpider
UrlProducer-3 called new htmlSpider
htmlSpider-0 got html text success in  17:54:22
htmlSpider-2 got html text success in  17:54:22
htmlSpider-1 got html text success in  17:54:22
UrlProducer-4 called new htmlSpider
UrlProducer-5 called new htmlSpider
UrlProducer-6 called new htmlSpider
htmlSpider-3 got html text success in  17:54:24
htmlSpider-4 got html text success in  17:54:24
UrlProducer-7 called new htmlSpider
htmlSpider-5 got html text success in  17:54:24
UrlProducer-8 called new htmlSpider
UrlProducer-9 called new htmlSpider
htmlSpider-6 got html text success in  17:54:26
htmlSpider-8 got html text success in  17:54:26
htmlSpider-7 got html text success in  17:54:26
UrlProducer-10 called new htmlSpider
UrlProducer-11 called new htmlSpider
UrlProducer-12 called new htmlSpider
htmlSpider-10 got html text success in  17:54:28
htmlSpider-11 got html text success in  17:54:28